# Gago juegos

## Codigo de blackbox

In [ ]:
import pandas as pd

def Filter(All_particles, particle_type, length_triangle):
    # Filter for charged particles
    filtered_particles = All_particles[All_particles['Particle Type'] == particle_type]
    
    # Filter for particles within the specified triangle length
    min_distance = length_triangle / 2
    max_distance = length_triangle * sqrt(3) / 2
    filtered_particles = filtered_particles[(filtered_particles['Detector 1'] >= min_distance) & (filtered_particles['Detector 1'] <= max_distance) &
                                           (filtered_particles['Detector 2'] >= min_distance) & (filtered_particles['Detector 2'] <= max_distance) &
                                           (filtered_particles['Detector 3'] >= min_distance) & (filtered_particles['Detector 3'] <= max_distance)]
    
    # Convert the resulting table to a dataframe
    df = pd.DataFrame(filtered_particles)
    
    return df

In [ ]:
import pandas as pd

def Filter(All_particles_df, particle_id, length_triangle):
    # Filter for particles with the specified ID
    filtered_df = All_particles_df[All_particles_df['ID'].isin(particle_id)]
    
    # Filter for particles within the specified length triangle
    center_detector = len(filtered_df) // 3 # Assume equal number of particles per detector
    x_min = center_detector - length_triangle
    x_max = center_detector + length_triangle
    filtered_df = filtered_df[(filtered_df['Detector'] >= x_min) & (filtered_df['Detector'] <= x_max)]
    
    return filtered_df

In [ ]:
import pandas as pd
import numpy as np

def Filter(All_particles_df, particle_id, length_triangle):
    # Filter for particles with the specified ID
    filtered_df = All_particles_df[All_particles_df['ID'].isin(particle_id)]

    # Define the coordinates of the triangle
    center_detector = len(filtered_df) // 3 # Assume equal number of particles per detector
    a = {'x': 0, 'y': 0}
    b = {'x': length_triangle, 'y': 0}
    c = {'x': center_detector, 'y': np.sqrt(3)/2 * length_triangle}

    # Calculate the distances of the particles to the triangle vertices
    ax = filtered_df['x'].values - a['x']
    ay = filtered_df['y'].values - a['y']
    bx = filtered_df['x'].values - b['x']
    by = filtered_df['y'].values - b['y']
    cx = filtered_df['x'].values - c['x']
    cy = filtered_df['y'].values - c['y']
    lengths = np.sqrt(ax**2 + ay**2), np.sqrt(bx**2 + by**2), np.sqrt(cx**2 + cy**2)

    # Filter for particles within the triangle
    filtered_df = filtered_df[np.any(np.all(lengths <= length_triangle, axis=1), axis=0)]

    return filtered_df

In [ ]:
import pandas as pd
import numpy as np

def Filter(All_particles_df, particle_id, length_triangle):
    # Filter for particles with the specified ID
    filtered_df = All_particles_df[All_particles_df['ID'].isin(particle_id)]

    # Define the coordinates of the triangle
    a = {'x': 0, 'y': 0}
    b = {'x': length_triangle, 'y': 0}
    c = {'x': length_triangle/2, 'y': np.sqrt(3)/2 * length_triangle}

    # Filter for particles within the triangle
    distances = np.array([np.linalg.norm(np.array(particle) - np.array(vertex)) for particle in filtered_df[['x', 'y']].values for vertex in [a, b, c]])
    filtered_df = filtered_df[distances.min(axis=1) <= length_triangle]

    return filtered_df

## Filtros

In [4]:
#Filtro geometrico
import numpy as np

def inside_detector(l, a, b, x, y):
    """
    Check if a point (x, y) is inside any of the detectors based on the triangle side length l
    and detector dimensions a and b.
    
    Parameters:
    l (float): the side of the triangle.
    a (float): the width of the detector.
    b (float): the height of the detector.
    x (float): x-coordinate of the point.
    y (float): y-coordinate of the point.
    
    Returns:
    int: The detector number if inside a detector, 0 otherwise.
    """
    
    # Check for D1
    if ((l / np.sqrt(3) - b / 2) <= x <= (l / np.sqrt(3) + b / 2)) and (-a / 2 <= y <= a / 2):
        return 1
    
    # Check for D2
    if ((-l / (2 * np.sqrt(3)) - b / 2) <= x <= (-l / (2 * np.sqrt(3)) + b / 2)) and ((l / 2 - a / 2) <= y <= (l / 2 + a / 2)):
        return 2
    
    # Check for D3
    if ((-l / (2 * np.sqrt(3)) - b / 2) <= x <= (-l / (2 * np.sqrt(3)) + b / 2)) and ((-l / 2 - a / 2) <= y <= (-l / 2 + a / 2)):
        return 3
    
    # If the point is not in any detector
    return 0

# Example to check if the function works
# We'll test with a triangle side length of 10, and detector dimensions 2x2
# We expect the following point to be in detector 1
l = 10
a = 2
b = 2
x_test = l / np.sqrt(3)
y_test = 0

inside_detector(l, a, b, x_test, y_test)


1

In [6]:
# Cargamos la lista de IDs desde el archivo
with open('particulas_cargadas.txt', 'r') as file:
    particle_ids = [int(line.strip()) for line in file.readlines()]

# Ahora definimos una función que filtrará las partículas basadas en el ID y si están dentro de un detector
def filter_particles(particles, l, a, b):
    """
    Filter the particles based on whether they are inside a detector and their ID is in the loaded list.
    
    Parameters:
    particles (list of tuples): List of tuples, where each tuple is (id, x, y), representing a particle.
    l (float): the side of the triangle.
    a (float): the width of the detector.
    b (float): the height of the detector.
    
    Returns:
    list: A list of particle IDs that are inside a detector and are in the allowed list.
    """
    filtered_ids = []
    for particle_id, x, y in particles:
        if particle_id in particle_ids and inside_detector(l, a, b, x, y):
            filtered_ids.append(particle_id)
    return filtered_ids

# Esta es una función de ejemplo que simula la entrada de partículas
def example_particle_stream(num_particles, l, a, b):
    """
    Generate a stream of particles with random IDs and positions.
    
    Parameters:
    num_particles (int): Number of particles to generate.
    l (float): the side of the triangle.
    a (float): the width of the detector.
    b (float): the height of the detector.
    
    Returns:
    list: A list of tuples representing particles with (id, x, y).
    """
    # Random state for reproducibility
    np.random.seed(0)
    particles = []
    for _ in range(num_particles):
        particle_id = np.random.randint(1, 200)  # Random ID between 1 and 200
        x = np.random.uniform(-l/2, l/2)  # Random x position within triangle bounds
        y = np.random.uniform(-l/(2*np.sqrt(3)), l*np.sqrt(3)/2)  # Random y position
        particles.append((particle_id, x, y))
    return particles

# Probemos con 100 partículas y veamos cuántas son aceptadas por el filtro.
example_particles = example_particle_stream(100, l, a, b)
filtered_particle_ids = filter_particles(example_particles, l, a, b)

# Ver los IDs de las partículas filtradas
filtered_particle_ids


[182]

## Codigo interesante

In [7]:
import pandas as pd
import numpy as np

# Vamos a simular un dataframe de pandas con las columnas 'ID', 'x' y 'y'.
# Esta sería la estructura del dataframe que recibiría la función 'Filter'.
# All_particles_df = pd.DataFrame({
#     'ID': [lista de IDs],
#     'x': [lista de coordenadas x],
#     'y': [lista de coordenadas y]
# })
l = 10
a = 2
b = 2
# Por ahora, creamos un dataframe de ejemplo con datos aleatorios para probar la función.
np.random.seed(0)  # Para reproducibilidad en el ejemplo
example_ids = np.random.randint(1, 200, size=100)
example_x = np.random.uniform(-l/2, l/2, size=100)
example_y = np.random.uniform(-l/(2*np.sqrt(3)), l*np.sqrt(3)/2, size=100)

All_particles_df = pd.DataFrame({
    'ID': example_ids,
    'x': example_x,
    'y': example_y
})

# Ahora definimos la función Filter.
def Filter(all_particles_df, charged_particles, length_triangle):
    """
    Apply geometric and charged particle ID filters to the dataframe of all particles.

    Parameters:
    all_particles_df (DataFrame): Dataframe with all particle data.
    charged_particles (list): List of charged particle IDs to filter.
    length_triangle (float): Length of the triangle side.

    Returns:
    DataFrame: Filtered dataframe with particle data that hit the detectors.
    """
    # Seleccionar las partículas que están en la lista de partículas cargadas
    charged_df = all_particles_df[all_particles_df['ID'].isin(charged_particles)]

    # Inicializar una lista para guardar los datos de las partículas detectadas
    detected_particles = []

    # Iterar sobre las partículas y aplicar el filtro de detector
    for index, row in charged_df.iterrows():
        detector_number = inside_detector(length_triangle, a, b, row['x'], row['y'])
        if detector_number:
            detected_particles.append((row['ID'], row['x'], row['y'], detector_number))

    # Crear un dataframe a partir de las partículas detectadas
    detected_df = pd.DataFrame(detected_particles, columns=['ID', 'x', 'y', 'Detector'])

    # Retornar el dataframe de partículas detectadas
    return detected_df

# Llamada de ejemplo a la función Filter
filtered_df = Filter(All_particles_df, particle_ids, l)
filtered_df.head()  # Mostrar las primeras filas del dataframe filtrado como ejemplo


,ID,x,y,Detector
0,194.0,-2.115236,4.245348,2
1,143.0,-3.587361,5.739779,2
2,54.0,4.840422,0.310542,1
3,24.0,4.825750,0.426258,1


In [8]:


# Un dataframe de ejemplo
df = pd.DataFrame({
    'A': [1, 2, 3],
    'B': [4, 5, 6]
})

# Usamos iterrows() para iterar sobre el dataframe
for index, row in df.iterrows():
    print("Índice:", index)
    print("Fila:", row)
    print("Valor en columna A:", row['A'])
    print("Valor en columna B:", row['B'])


Índice: 0
Fila: A    1
B    4
Name: 0, dtype: int64
Valor en columna A: 1
Valor en columna B: 4
Índice: 1
Fila: A    2
B    5
Name: 1, dtype: int64
Valor en columna A: 2
Valor en columna B: 5
Índice: 2
Fila: A    3
B    6
Name: 2, dtype: int64
Valor en columna A: 3
Valor en columna B: 6


## The real goated

In [11]:
import pandas as pd
import numpy as np

def Filter_boolean(all_particles_df, charged_particles, length_triangle, a, b):
    """
    Apply geometric and charged particle ID filters to the dataframe of all particles using vectorized operations.
    Adjusted to include only particle ID and time in the output for the provided data structure.
    
    Parameters:
    all_particles_df (pd.DataFrame): Dataframe with all particle data, including 'id', 'x', 'y', and 't' columns.
    charged_particles (list): List of charged particle IDs to filter.
    length_triangle (float): Length of the triangle side.
    a (float): The width of the detector.
    b (float): The height of the detector.
    
    Returns:
    pd.DataFrame: Filtered dataframe with only the ID and time of particles that hit the detectors.
    """
    # Crear una máscara booleana para los IDs de partículas cargadas
    mask_charged = all_particles_df['id'].isin(charged_particles)

    # Calcular las máscaras booleanas para cada detector
    mask_d1 = ((all_particles_df['x'] >= (length_triangle / np.sqrt(3) - b / 2)) & 
               (all_particles_df['x'] <= (length_triangle / np.sqrt(3) + b / 2)) & 
               (all_particles_df['y'] >= -a / 2) & 
               (all_particles_df['y'] <= a / 2))
    
    mask_d2 = ((all_particles_df['x'] >= (-length_triangle / (2 * np.sqrt(3)) - b / 2)) & 
               (all_particles_df['x'] <= (-length_triangle / (2 * np.sqrt(3)) + b / 2)) & 
               (all_particles_df['y'] >= (length_triangle / 2 - a / 2)) & 
               (all_particles_df['y'] <= (length_triangle / 2 + a / 2)))
    
    mask_d3 = ((all_particles_df['x'] >= (-length_triangle / (2 * np.sqrt(3)) - b / 2)) & 
               (all_particles_df['x'] <= (-length_triangle / (2 * np.sqrt(3)) + b / 2)) & 
               (all_particles_df['y'] >= (-length_triangle / 2 - a / 2)) & 
               (all_particles_df['y'] <= (-length_triangle / 2 + a / 2)))

    # Aplicar la máscara de IDs cargados y las máscaras de detectores
    all_particles_df['Detector'] = 0
    all_particles_df.loc[mask_charged & mask_d1, 'Detector'] = 1
    all_particles_df.loc[mask_charged & mask_d2, 'Detector'] = 2
    all_particles_df.loc[mask_charged & mask_d3, 'Detector'] = 3

    # Filtrar las filas donde 'Detector' es diferente de 0 y seleccionar solo las columnas 'id' y 't'
    filtered_df = all_particles_df[all_particles_df['Detector'] != 0][['id', 't', 'Detector']].reset_index(drop=True)
    df_detector_1 = filtered_df[filtered_df['Detector'] == 1].reset_index(drop=True)
    df_detector_2 = filtered_df[filtered_df['Detector'] == 2].reset_index(drop=True)
    df_detector_3 = filtered_df[filtered_df['Detector'] == 3].reset_index(drop=True)

    return df_detector_1, df_detector_2, df_detector_3
